In [1]:
import os
import pickle
import numpy as np
import scipy.sparse as sp
import networkx
import importlib
import torch
import torch.nn.functional as F

In [2]:
import sys
sys.path.append('..')

import grb
import grb.utils as utils
from grb.utils import evaluator

# Load data

## Pre-defined dataset

In [35]:
from grb.dataset.dataset import Dataset
dataset = Dataset('cora')

Dataset 'cora' loaded.
    Number of nodes: 2708.
    Number of edges: 5242.
    Number of features: 1433.
    Number of classes: 7.
    Number of train samples: 140.
    Number of val samples: 500.
    Number of test samples: 1000.
    Feature range: [0.0000, 1.0000]


## Custom dataset

In [4]:
data_dir = "../data/Refined-cora-citeseer"
with open(os.path.join(data_dir, "corax_adj.pkl"), 'rb') as f:
    adj = pickle.load(f)
with open(os.path.join(data_dir, "corax_features.pkl"), 'rb') as f:
    features = pickle.load(f)
with open(os.path.join(data_dir, "corax_labels.pkl"), 'rb') as f:
    labels = pickle.load(f)
    labels = np.argmax(labels, axis=1)
    
n_node = features.shape[0]
train_mask = torch.zeros(n_node, dtype=bool)
train_mask[range(1180)] = True
val_mask = torch.zeros(n_node, dtype=bool)
val_mask[range(1180, 2180)] = True
test_mask = torch.zeros(n_node, dtype=bool)
test_mask[range(2180, 2680)] = True

In [5]:
from grb.dataset.dataset import CustomDataset

dataset = CustomDataset(adj=adj,
                        features=features,
                        labels=labels,
                        train_mask=train_mask,
                        val_mask=val_mask,
                        test_mask=test_mask,
                        name='Refined-cora')

Custom Dataset 'Refined-cora' loaded.
    Number of nodes: 2680.
    Number of edges: 10296.
    Number of features: 302.
    Number of classes: 7.
    Number of train samples: 1180.
    Number of val samples: 1000.
    Number of test samples: 500.
    Feature range [-2.2968, 2.4000]


## Load data

In [36]:
device = 'cuda:3'
adj = dataset.adj
adj_tensor = dataset.adj_tensor
features = dataset.features
labels = dataset.labels
num_features = dataset.num_features
num_classes = dataset.num_classes

# Prepare surrogate model

## Build model

In [37]:
from grb.model.gcn import GCN
from grb.utils.normalize import GCNAdjNorm

In [38]:
model = GCN(in_features=num_features, 
            out_features=num_classes, 
            hidden_features=[64, 64], 
            activation=F.relu)
print("Number of parameters: {}.".format(utils.get_num_params(model)))
print(model)

Number of parameters: 96391.
GCN(
  (layers): ModuleList(
    (0): GCNConv(
      (linear): Linear(in_features=1433, out_features=64, bias=True)
    )
    (1): GCNConv(
      (linear): Linear(in_features=64, out_features=64, bias=True)
    )
    (2): GCNConv(
      (linear): Linear(in_features=64, out_features=7, bias=True)
    )
  )
)


## Model training

In [39]:
from grb.model.trainer import Trainer

adam = torch.optim.Adam(model.parameters(), lr=0.01)
trainer = Trainer(dataset=dataset, 
                  optimizer=adam, 
                  loss=F.nll_loss,
                  adj_norm_func=GCNAdjNorm,
                  lr_scheduler=False,
                  early_stop=False,
                  device=device)

In [40]:
trainer.train(model=model,
              n_epoch=1000,
              save_dir="../saved_models/gcn_cora",
              eval_every=50,
              save_after=500,
              dropout=0.5,
              verbose=True)

Epoch 00000 | Train Loss 1.9546 | Train Acc 0.1214 | Val Loss 1.9551 | Val Acc 0.1080
Epoch 00050 | Train Loss 0.0223 | Train Acc 1.0000 | Val Loss 1.0466 | Val Acc 0.7320
Epoch 00100 | Train Loss 0.0035 | Train Acc 1.0000 | Val Loss 1.2865 | Val Acc 0.7540
Epoch 00150 | Train Loss 0.0012 | Train Acc 1.0000 | Val Loss 1.4228 | Val Acc 0.7440
Epoch 00200 | Train Loss 0.0015 | Train Acc 1.0000 | Val Loss 1.4812 | Val Acc 0.7680
Epoch 00250 | Train Loss 0.0004 | Train Acc 1.0000 | Val Loss 1.7176 | Val Acc 0.7480
Epoch 00300 | Train Loss 0.0002 | Train Acc 1.0000 | Val Loss 1.6525 | Val Acc 0.7400
Epoch 00350 | Train Loss 0.0009 | Train Acc 1.0000 | Val Loss 1.7108 | Val Acc 0.7540
Epoch 00400 | Train Loss 0.0014 | Train Acc 1.0000 | Val Loss 1.6388 | Val Acc 0.7560
Epoch 00450 | Train Loss 0.0007 | Train Acc 1.0000 | Val Loss 1.7851 | Val Acc 0.7460
Epoch 00500 | Train Loss 0.0002 | Train Acc 1.0000 | Val Loss 1.9398 | Val Acc 0.7420
Epoch 00550 | Train Loss 0.0002 | Train Acc 1.0000 | V

## Load trained weights

In [9]:
model.load_state_dict(torch.load('../saved_models/gcn_cora/checkpoint.pt'))

<All keys matched successfully>

## Evaluation

In [10]:
pred = model.forward(features, utils.adj_preprocess(adj, adj_norm_func=GCNAdjNorm), dropout=0)
pred_label = torch.argmax(pred, dim=1)
acc = evaluator.eval_acc(pred, labels, mask=dataset.test_mask)

print("Test accuracy: {:.4f}".format(acc))

Test accuracy: 0.8400


# Attack

## FGSM

In [27]:
from grb.attack.fgsm import FGSM

config = {}
config['epsilon'] = 0.01
config['n_epoch'] = 10
config['feat_lim_min'] = 0
config['feat_lim_max'] = 1
config['n_inject_max'] = 100
config['n_edge_max'] = 20

fgsm = FGSM(dataset, adj_norm_func=GCNAdjNorm, device=device)
fgsm.set_config(**config)
adj_attack, features_attack = fgsm.attack(model)

Epoch 0, Loss: 3.04357, Test acc: 0.75000
Epoch 1, Loss: 3.32545, Test acc: 0.63200
Epoch 2, Loss: 3.48930, Test acc: 0.51000
Epoch 3, Loss: 3.56440, Test acc: 0.44000
Epoch 4, Loss: 3.71458, Test acc: 0.41800
Epoch 5, Loss: 3.90337, Test acc: 0.41600
Epoch 6, Loss: 4.08175, Test acc: 0.38400
Epoch 7, Loss: 4.28346, Test acc: 0.39600
Epoch 8, Loss: 4.50823, Test acc: 0.37000
Epoch 9, Loss: 4.68984, Test acc: 0.38600


In [26]:
importlib.reload(grb.attack.fgsm)

<module 'grb.attack.fgsm' from '../grb/attack/fgsm.py'>

## SPEIT

In [13]:
from grb.attack.speit import SPEIT

config = {}
config['inject_mode'] = 'random-inter'
config['lr'] = 0.01
config['n_epoch'] = 100
config['feat_lim_min'] = 0
config['feat_lim_max'] = 1
config['n_inject_max'] = 100
config['n_edge_max'] = 20

speit = SPEIT(dataset, adj_norm_func=GCNAdjNorm, device=device)
speit.set_config(**config)
target_node = np.random.choice(dataset.num_test, 100)
adj_attack, features_attack = speit.attack(model, target_node)

Epoch 0, Loss: 22.49386, Test acc: 0.99600
Epoch 1, Loss: 22.39210, Test acc: 0.99600
Epoch 2, Loss: 22.27702, Test acc: 0.99600
Epoch 3, Loss: 22.14484, Test acc: 0.98600
Epoch 4, Loss: 21.99894, Test acc: 0.97600
Epoch 5, Loss: 21.84322, Test acc: 0.97200
Epoch 6, Loss: 21.68154, Test acc: 0.95600
Epoch 7, Loss: 21.51576, Test acc: 0.95000
Epoch 8, Loss: 21.34705, Test acc: 0.93800
Epoch 9, Loss: 21.17599, Test acc: 0.93000
Epoch 10, Loss: 21.00336, Test acc: 0.91800
Epoch 11, Loss: 20.82933, Test acc: 0.90800
Epoch 12, Loss: 20.65326, Test acc: 0.89800
Epoch 13, Loss: 20.47496, Test acc: 0.89400
Epoch 14, Loss: 20.29387, Test acc: 0.88600
Epoch 15, Loss: 20.11124, Test acc: 0.87800
Epoch 16, Loss: 19.92774, Test acc: 0.87400
Epoch 17, Loss: 19.74131, Test acc: 0.86600
Epoch 18, Loss: 19.55345, Test acc: 0.86400
Epoch 19, Loss: 19.36393, Test acc: 0.85600
Epoch 20, Loss: 19.17199, Test acc: 0.85400
Epoch 21, Loss: 18.97903, Test acc: 0.84600
Epoch 22, Loss: 18.78416, Test acc: 0.8460

In [14]:
utils.save_features(features_attack, './speit_cora', 'features.npy')
utils.save_adj(adj_attack, './speit_cora', 'adj.pkl')

## TDGIA

In [53]:
from grb.attack.tdgia import TDGIA

config = {}
config['inject_mode'] = 'random'
config['lr'] = 0.01
config['n_epoch'] = 100
config['feat_lim_min'] = 0
config['feat_lim_max'] = 1
config['n_inject_max'] = 100
config['n_edge_max'] = 20

tdgia = TDGIA(dataset, adj_norm_func=GCNAdjNorm, device=device)
tdgia.set_config(**config)
adj_attack, features_attack = tdgia.attack(model)

Epoch 0, Loss: 11.39142, Test acc: 0.71200
Epoch 1, Loss: 3.80843, Test acc: 0.23800
Epoch 2, Loss: 3.18443, Test acc: 0.19900
Epoch 3, Loss: 3.05643, Test acc: 0.19100
Epoch 4, Loss: 2.97643, Test acc: 0.18600
Epoch 5, Loss: 2.96043, Test acc: 0.18500
Epoch 6, Loss: 2.96043, Test acc: 0.18500
Epoch 7, Loss: 2.96043, Test acc: 0.18500
Epoch 8, Loss: 2.96043, Test acc: 0.18500
Epoch 9, Loss: 2.96043, Test acc: 0.18500
Epoch 10, Loss: 2.94443, Test acc: 0.18400
Epoch 11, Loss: 2.94443, Test acc: 0.18400
Epoch 12, Loss: 2.92845, Test acc: 0.18300
Epoch 13, Loss: 2.91243, Test acc: 0.18200
Epoch 14, Loss: 2.91243, Test acc: 0.18200
Epoch 15, Loss: 2.89643, Test acc: 0.18100
Epoch 16, Loss: 2.89642, Test acc: 0.18100
Epoch 17, Loss: 2.88043, Test acc: 0.18000
Epoch 18, Loss: 2.88043, Test acc: 0.18000
Epoch 19, Loss: 2.86443, Test acc: 0.17900
Epoch 20, Loss: 2.86443, Test acc: 0.17900
Epoch 21, Loss: 2.86443, Test acc: 0.17900
Epoch 22, Loss: 2.86443, Test acc: 0.17900
Epoch 23, Loss: 2.84